# Get the model metadata from ModelMesh Server via GRPC

#### Set grpc endpoint and model name

In [ ]:
!pip install grpcio grpcio-tools 

In [ ]:
grpc_host = "modelmesh-serving"
grpc_port = 8033
model_name = "stocks"

#### Imports

In [ ]:
# grpc_predict_v2_pb2 and grpc_predict_v2_pb2_grpc were created from grpc_predict_v2.proto using protoc
import grpc
import grpc_predict_v2_pb2
import grpc_predict_v2_pb2_grpc

#### Create grpc channel and stub

In [ ]:
channel = grpc.insecure_channel(f"{grpc_host}:{grpc_port}")
stub = grpc_predict_v2_pb2_grpc.GRPCInferenceServiceStub(channel)

### Model Metadata
The per-model metadata API provides information about a model. Errors are indicated by the google.rpc.Status returned for the request. The OK code indicates success and other codes indicate failure.

In [ ]:
request = grpc_predict_v2_pb2.ModelMetadataRequest(name=model_name)
response = stub.ModelMetadata(request)
print(response)

In [ ]:
import time

def rhods_grpc_request(data):
    # request content building
    inputs = []
    inputs.append(grpc_predict_v2_pb2.ModelInferRequest().InferInputTensor())
    inputs[0].name = "lstm_input"
    inputs[0].datatype = "FP32"
    inputs[0].shape.extend([69, 60, 1])
    arr = data.flatten()
    inputs[0].contents.fp32_contents.extend(arr)

    # request building
    request = grpc_predict_v2_pb2.ModelInferRequest()
    request.model_name = model_name
    request.inputs.extend(inputs)

    response = stub.ModelInfer(request)    
    result_arr = np.frombuffer(response.raw_output_contents[0], dtype=np.float32)
    return result_arr

In [ ]:
import numpy as np

data = np.random.uniform(low=100.1, high=125.1, size=(69, 60, 1))
predicted_stock_price = rhods_grpc_request(data)
predicted_stock_price

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(
    predicted_stock_price, color="red", label="Predicted Stock Price"
)
plt.title("Stock Price Prediction")
plt.xlabel("Date")
plt.ylabel("Stock Price")
plt.legend()
plt.show()